In [560]:
from bs4 import BeautifulSoup
import os
import parser,urllib.request,requests
from lxml import html,etree
import re
import pandas as pd

download the html

In [580]:
opener = urllib.request.FancyURLopener({})
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
f = opener.open(url)
soup = BeautifulSoup(f, 'lxml')

C:\Users\Daniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


locate the table within the html

In [581]:
content = []
for td in soup.table.tbody.find_all('td'):
    content.append(td.text)

In [790]:
content = [i.strip('\n') for i in content]

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

rows were divided by the structure of the table. 3 indicates that the list content will be divided every third elements from the index 0.

Boroughs are divided starting from the index 1 and Neighborhood will be dvided from the third element and so forth.

In [722]:

rows = {
    'PostalCode':content[::3],
    'Borough':content[1::3],
    'Neighborhood':content[2::3]
    
}
df = pd.DataFrame(rows)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [723]:
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True,inplace=True)

These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

The neighborhood will be combined in accordance to postalcode and borough (grouped).

In [724]:
df = df.groupby(['PostalCode','Borough']).Neighborhood.agg([('Neighborhood',', '.join)])
df.reset_index(inplace=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

This basically pinpoint where neighborhood content is 'Not assigned' and copy/paste its borough content into it.

In [786]:
df.loc[df[df['Neighborhood']=='Not assigned']['Neighborhood'].index.values[0]]['Neighborhood'] = df.loc[df[df['Neighborhood']=='Not assigned']['Neighborhood'].index.values[0]]['Borough']

In [788]:
df.shape

(103, 3)